In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
# import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

In [4]:

model = models.resnet50(weights= None)

In [5]:
# print(model)

In [6]:
model.fc = nn.Linear(2048,50)

In [7]:
import torchinfo

torchinfo.summary(model, (32, 3,64,64))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [32, 50]                  --
├─Conv2d: 1-1                            [32, 64, 32, 32]          9,408
├─BatchNorm2d: 1-2                       [32, 64, 32, 32]          128
├─ReLU: 1-3                              [32, 64, 32, 32]          --
├─MaxPool2d: 1-4                         [32, 64, 16, 16]          --
├─Sequential: 1-5                        [32, 256, 16, 16]         --
│    └─Bottleneck: 2-1                   [32, 256, 16, 16]         --
│    │    └─Conv2d: 3-1                  [32, 64, 16, 16]          4,096
│    │    └─BatchNorm2d: 3-2             [32, 64, 16, 16]          128
│    │    └─ReLU: 3-3                    [32, 64, 16, 16]          --
│    │    └─Conv2d: 3-4                  [32, 64, 16, 16]          36,864
│    │    └─BatchNorm2d: 3-5             [32, 64, 16, 16]          128
│    │    └─ReLU: 3-6                    [32, 64, 16, 16]          --
│ 

In [8]:
train_dir = '/kaggle/input/iith-dl-contest-2024/train/train'
train_data  = datasets.ImageFolder(train_dir, transform = transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [9]:
model = model.load_state_dict(torch.load(/kaggle/input/resnet50/weights16.pth))

SyntaxError: invalid syntax (2863026206.py, line 1)

In [ ]:
test_dir = '/kaggle/input/iith-dl-contest-2024/test'
test_data  = datasets.ImageFolder(test_dir, transform = transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
import numpy as np
import csv

In [ ]:
# Reverse the class to index mapping to index to class for prediction interpretation
classes = train_data.class_to_idx
idx_to_class = {idx: class_name for class_name, idx in classes.items()}

In [ ]:
# List to store outputs
outputs_list = []

# Process images and gather predictions
for images, _ in tqdm(test_loader):
    images = images.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, dim=1)
    outputs_list.append(predicted)

# Concatenate all predictions into a single tensor
outputs = torch.cat(outputs_list).cpu().numpy()

In [ ]:
import numpy as np

# Convert class indices to class names
predicted_classes = np.array([idx_to_class[idx] for idx in outputs], dtype=object)

# Generate image file names
file_names = [f"{i}.JPEG" for i in range(len(predicted_classes))]

file_names = np.sort(file_names)

In [ ]:
# Combine file names and predicted classes
table = np.column_stack((file_names, predicted_classes))

# Print table (optional, can be commented out in production)
print(table)

In [ ]:
# Write results to CSV file
with open('submission1.csv', 'w', newline='') as file:
    wr = csv.writer(file)
    wr.writerow(['ID', 'Category'])
    wr.writerows(table)